In [1]:
import pandas as pd
import glob
from datetime import datetime

In [2]:
# SDG&E rates
RATES = {
    'summer': {'on_peak': 0.59908, 'off_peak': 0.52754, 'super_off_peak': 0.45000},
    'winter': {'on_peak': 0.58155, 'off_peak': 0.51899, 'super_off_peak': 0.50084}
}

HOLIDAYS = ['1/1/2025', '2/17/2025', '5/26/2025', '7/4/2025', '9/1/2025', '11/11/2025', '11/27/2025', '12/25/2025']

In [3]:
def process_gbd_file(filepath):
    # Find where data starts
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    header_row = 0
    for i, line in enumerate(lines):
        if 'Meter Number,Date,Start Time' in line:
            header_row = i
            break
    
    # Read data
    df = pd.read_csv(filepath, skiprows=header_row)
    
    # Get account info
    account_info = {
        'name': lines[0].split(',')[1].strip(),
        'meter_number': lines[6].split(',')[1].strip()
    }
    
    # Process timestamps
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Start Time'])
    df['Hour'] = df['DateTime'].dt.hour
    df['Month'] = df['DateTime'].dt.month
    df['Weekday'] = df['DateTime'].dt.dayofweek
    
    # Determine season
    df['Season'] = df['Month'].apply(lambda m: 'summer' if 6 <= m <= 10 else 'winter')
    
    # Determine if weekend/holiday
    df['IsWeekendHoliday'] = (df['Weekday'] >= 5) | (df['DateTime'].dt.strftime('%-m/%-d/%Y').isin(HOLIDAYS))
    
    # Determine time period
    def get_period(row):
        hour = row['Hour']
        if row['IsWeekendHoliday']:
            if 16 <= hour < 21: return 'on_peak'
            elif 14 <= hour < 16 or 21 <= hour: return 'off_peak'
            else: return 'super_off_peak'
        else:
            if 16 <= hour < 21: return 'on_peak'
            elif 6 <= hour < 16 or 21 <= hour:
                if row['Month'] in [3,4] and 10 <= hour < 14: return 'super_off_peak'
                return 'off_peak'
            else: return 'super_off_peak'
    
    df['TimePeriod'] = df.apply(get_period, axis=1)
    
    # Apply rates
    df['Rate'] = df.apply(lambda row: RATES[row['Season']][row['TimePeriod']], axis=1)
    df['Cost'] = df['Net'].astype(float) * df['Rate']
    
    # Add account info
    for k, v in account_info.items():
        df[k] = v
    
    return df

In [4]:
# Process all files
all_data = []

for csv_file in glob.glob('./gbd_data/*.csv'):
    print(f"Processing {csv_file}")
    df = process_gbd_file(csv_file)
    all_data.append(df)
    print(f"  {len(df)} rows, ${df['Cost'].sum():.2f} total")

# Combine and save
if all_data:
    combined = pd.concat(all_data)
    combined.to_csv('./processed_sdge_data.csv', index=False)
    print(f"\nSaved {len(combined)} total rows")

Processing ./gbd_data/Electric_15_Minute_512025_5302025_20250609.csv


/tmp/ipykernel_2409/2691603573.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Start Time'])


  2880 rows, $16.08 total

Saved 2880 total rows
